In [1]:
import os
import itk                                                                
import itkwidgets
import numpy as np
import pandas as pd
import nibabel as nib
import seaborn as sns
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, IntSlider, ToggleButtons

sns.set_style('darkgrid')

In [2]:
df = pd.read_csv("NFBS_Dataset_meta.csv")

In [3]:
def get_image_slider(row):
    images = []

    for file in row:
        image = nib.load(file).get_fdata()
        images.append(np.expand_dims(image, axis=3))
    
    def explore_3dimage(depth):
        plt.figure(figsize=(10, 5))
        labels = ["MRI - Skull Layers", "Skull Stripped Brain Layers", "Mask Layers"]
        for i, image in enumerate(images):
            plt.subplot(1, len(images), i+1)
            plt.imshow(image[:, :, depth, :], cmap='gray')
            plt.title(labels[i])
            plt.axis('off')

    interact(explore_3dimage, depth=(0, images[0].shape[2] - 1));

In [4]:
get_image_slider(df.loc[0])

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…

In [5]:
get_image_slider(df.loc[10])

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…

In [6]:
get_image_slider(df.loc[124])

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…

In [7]:
def z_score(img, mask=None):
    img_data = img.get_fdata()
    if mask is not None and not isinstance(mask, str):
        mask_data = mask.get_fdata()
    else:
        mask_data = img_data == img_data
    logical_mask = mask_data > 0.  
    mean = img_data[logical_mask].mean()
    std = img_data[logical_mask].std()
    normalized = nib.Nifti1Image((img_data - mean) / std, img.affine, img.header)
    return normalized

In [8]:
def visualize_norm_and_unnorm_images(img, mask=None, preprocess=z_score):
    img = nib.load(img)
    mask = nib.load(mask) if mask is not None else None

    norm_img = np.expand_dims(preprocess(img, mask).get_fdata(), axis=3)
    img = np.expand_dims(img.get_fdata(), axis=3)

    def explore_3dimage(depth):
        plt.figure(figsize=(10, 5))
        
        plt.subplot(1, 2, 1)
        plt.imshow(img[:, :, depth, :], cmap='gray')
        plt.title("Unnormalized Image")
        plt.axis('off')

        plt.subplot(1, 2, 2)
        plt.imshow(norm_img[:, :, depth, :], cmap='gray')
        plt.title("Intensity Normalized Image")

        plt.axis('off')

    interact(explore_3dimage, depth=(0, img.shape[2] - 1));

In [9]:
visualize_norm_and_unnorm_images(df['skull'][0])

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…

In [10]:
from intensity_normalization.normalize.kde import kde_normalize
visualize_norm_and_unnorm_images(df['skull'][0], preprocess=kde_normalize)

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…

In [11]:
def plot_single_image(img):
    img = nib.load(img)
    
    def explore_3dimage(depth):
        plt.figure(figsize=(10, 5))
        
        plt.subplot(1, 1, 1)
        plt.imshow(np.expand_dims(img.get_fdata(), axis=3)[:, :, depth, :], cmap='gray')
        plt.title("MRI")
        plt.axis('off')

    interact(explore_3dimage, depth=(0, img.shape[2] - 1));

In [12]:
plot_single_image('T1Img/sub-01/T1w.nii.gz')

interactive(children=(IntSlider(value=127, description='depth', max=255), Output()), _dom_classes=('widget-int…

In [13]:
plot_single_image('T1Img/sub-02/anat_img.nii.gz')

interactive(children=(IntSlider(value=127, description='depth', max=255), Output()), _dom_classes=('widget-int…

In [14]:
plot_single_image(df['skull'][0])

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…

In [15]:
plot_single_image(df['brain'][124])

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…

In [16]:
plot_single_image('T1Img/sub-05/T1w MRI.nii')

interactive(children=(IntSlider(value=82, description='depth', max=165), Output()), _dom_classes=('widget-inte…

In [17]:
plot_single_image('mask.nii.gz')

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…

In [18]:
plot_single_image('bias_corrected_image.nii.gz')

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…

In [19]:
plot_single_image(df['skull'][0])

interactive(children=(IntSlider(value=95, description='depth', max=191), Output()), _dom_classes=('widget-inte…